In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
try:
    os.chdir("./drive/My Drive/RuCode3/")
    #                 ^  ваш путь к папке с данными
except Exception as e:
    print(e)

Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x
#                    ^ version 1.15.2
# https://github.com/WindQAQ/tf-recsys
# ^ ссылка на библиотеку, есть баги и об этом ниже

# https://github.com/WindQAQ/tf-recsys/pull/2/commits
# ^ ссылка на фиксы

# на всякий случай пользуйтесь нашей версией в папке
from tf_recsys.tfcf.metrics import mae, rmse
from tf_recsys.tfcf.config import Config
from tf_recsys.tfcf.models.svd import SVD
from tf_recsys.tfcf.models.svdpp import SVDPP
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import pandas as pd
import random

np.random.seed(42)
random.seed(42)
tf.set_random_seed(42)
 
ratings = pd.read_csv("user_ratings.csv", dtype=np.int32)
ratings = ratings.loc[ratings["rating"] != 0]
x = ratings[["user_id", "anime_id"]].values
y = ratings["rating"].values

test_size = 0.01
#             ^ ~100k сэмплов(мало, но нам хватит)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=0)
 
config = Config()
config.num_users = np.max(x[:, 0]) + 1
config.num_items = np.max(x[:, 1]) + 1
config.min_value = np.min(y)
config.max_value = np.max(y)
config.num_factors = 42
config.reg_p_u = 0.01
config.reg_q_i = 0.01
config.reg_y_u = 0.01
config.reg_g_i = 0.01

epochs = 4 # +- час на итерацию с использованием Tesla K80(T4)
batch_size = 8192

with tf.Session() as sess:
    tf.set_random_seed(42)
    model = SVDPP(config, sess, dual=True)

    print("Start train")
    model.train(x_train,
                y_train,
                validation_data=(x_test, y_test),
                epochs=epochs,
                batch_size=batch_size)

    y_pred = model.predict(x_test)
    print('rmse: {}, mae: {}'.format(rmse(y_test, y_pred), mae(y_test, y_pred)))
 
    model.save_model(f'model_{epochs}it_{batch_size}_num_factors{config.num_factors}_test_size{test_size}_red_all_{config.reg_p_u}/')

    sample = pd.read_csv("sample_submission.csv")
    sample_new = np.array([[int(i) for i in id.split()] for id, rat in sample.values])
    test_preds = model.predict(sample_new)
    sample["rating"] = pd.Series([float(x) for x in test_preds])
    sample.to_csv(f"tfcf_sub_model_{epochs}it_{batch_size}_num_factors{config.num_factors}_test_size{test_size}_red_all_{config.reg_p_u}.csv",
                  index=None)

TensorFlow 1.x selected.
Start train



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



Epoch 1/4
1318/1318 [==============================] - 2959s 2s/step - rmse: 1.2432 - mae: 0.9302 - val_rmse: 1.2002 - val_mae: 0.8972
Epoch 2/4
1318/1318 [==============================] - 2857s 2s/step - rmse: 1.1450 - mae: 0.8507 - val_rmse: 1.1686 - val_mae: 0.8702
Epoch 3/4
Epoch 3/4
1318/1318 [==============================] - 2648s 2s/step - rmse: 1.0951 - mae: 0.8119 - val_rmse: 1.1564 - val_mae: 0.8604
Epoch 4/4
Epoch 4/4
1318/1318 [==============================] - 2590s 2s/step - rmse: 1.0551 - mae: 0.7809 - val_rmse: 1.1510 - val_mae: 0.8547
rmse: 1.151020651447436, mae: 0.854725438320983

rmse: 1.151020651447436, mae: 0.854725438320983

